## **SISSEJUHATUS ERIALASSE ANDMETÖÖTLUSE KODUTÖÖ:**

# **WINE REVIEWS**

### *Merilin Kõrnas ja Õie Renata Siimon*

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://img1.goodfon.com/wallpaper/big/8/b7/vinograd-korzina-vetki-listya.jpg")

## **1. ANDMED:**
Andmetena kasutasime Kaggle'ist saadud Wine Reviews andmebaasi[1], kus on toodud degustaatorite hinnangud veinidele erinevate veinisortide ja tootmispiirkondade lõikes koos veini hindadega. 

**
[1] Kaggle, "Wine Reviews" https://www.kaggle.com/zynicide/wine-reviews [04.12.2017]

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pycountry
import matplotlib.pyplot as plt
import matplotlib.colors

#from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

#Andmete sisselugemine:
df = pd.read_csv("../input/winemag-data_first150k.csv", encoding="utf-8")
df

> ## **2. KIRJELDAV STATISTIKA:**

In [ ]:
#Vaatame andmebaasi kirjeldavat statistikat:
df.info()

Nagu kirjeldavast statistikast näha, sisaldab andmestik hinnanguid veidi rohkem kui 150 000 veinile. 

Lisaks veinisordile *('variety')* ja riigile *('country')* on andmebaasis info ka täpsema päritolu piirkonna *('province', 'region_1'* ja *'region_2')* (nt California või Burgundia), tootja nime *('winery')* ja kaubamärgi *('designation')* kohta ning veini maitseomaduste kirjeldus *('description')* . 

Punktihinnang on antud täisarvuna ja hind ujukomaarvuna, ülejäänud muutujad on sõned.

Tabelis on ka puuduvaid väärtusi, sh meid huvitavates veergudes 'points' ja 'price', kuid analüüsi käigus selgus, et need ei osutunud siiski probleemiks, sest pandas oskas need ise välja jätta.

Andmestiku puuduseks on see, et puudb info veinide aastakäikude kohta, mis ometi on veini hinda ja kvaliteeti oluliselt mõjutav tegur.

## **3. UURIMISKÜSIMUSED:**

Soovisime leida andmebaasist vastused järgmistele küsimustele:

**1) Milline on veinide kvaliteet erinevates riikides?**

**2) Milline on veinide keskmine hind erinevates riikides?**

**3) Miline on veinide keskmine hinna/kvaliteedi suhe erinevates riikides?**


## **4. ANALÜÜS:**
## **4.1 Milline on veinide kvaliteet erinevates riikides?**
Veinide kvaliteedi uurimiseks vaatame, millised on degustaatorite keskmised hinnangud erinevate riikide veinidele. Lisaks vaatame ka, millise riigi veinid on saanud degustaatoritelt kõige kõrgemaid/madalamaid hinnangud.

Kuvame kõigepealt tabelist riigi ja punktide veerud, kus iga riigi kohta on toodud keskmine hinnang, mille degustaatorid on selle riigi veinidele andnud, ning sorteerime tabeli punktide veeru järgi kahanevas järjekorras:

In [ ]:
df2=pd.DataFrame(df.groupby("country")["points"].mean())
df2.sort_values("points", ascending=False)

Nagu tabelist näeme, on degustaatorid andnud keskmiselt kõige parema hinnangu Inglismaa ja kõige halvema hinnangu Lõuna-Korea veinidele. Samas võib täheldada, et vahe minimaalse ja maksimaalse skoori saanud riigi vahel ei ole väga suur (vaid 11 punkti). 

Seetõttu leidsime, et huvitav oleks vaadata lisaks riikide keskmisele skoorile ka minimaalset ja maksimaalset skoori ning seda, millises vahemikus skoor kõigub. Viimase nägemiseks leiame riigi maksimaalse ja minimaalse skoori vahe ning riigi skooride standardhälbe (standardhälve iseloomustab keskmist erinevust keskmisest). Et saadud näitajad informatiivsemad oleksid, lisame tabelisse ka selle, mitu hinnangut iga riigi veinidele üldse antud on.


In [ ]:
#Punktiskoori veergude lisamine:
df2["max_points"]=pd.DataFrame(df.groupby("country")["points"].max())
df2["min_points"]=pd.DataFrame(df.groupby("country")["points"].min())
df2["max-min_points"]=df2["max_points"]-df2["min_points"] #lahutame ühe Series objekti teisest
df2["stdev_points"]=pd.DataFrame(df.groupby("country")["points"].std())
df2["hinnanguid"]=df["country"].value_counts()

df2.sort_values("points", ascending=False)

Nagu  tabelist näha, on hinnangute arv riikide lõikes erinev. Riikide puhul, millele on antud vaid paar hinnangut, ei saa kvaliteedistatistkiat aga usaldusväärseks pidada, sest see põhineb vaid paarile selle riigi veinile antud hinnangul. Seetõttu jätame tabelist välja väga vähesel arvul hinnanguid (alla 20 hinnangu) saanud riigid.

Selleks, et riigid tulpdiagrammile ära mahuksid, lisame tabelisse ka riikide ISO-koodid.

In [ ]:
#Filtreerime:
df3=df2[df2.hinnanguid>=20]

#Riikide ISO-koodide lisamine: enamiku ISO-koode saab kätte pythoni funktsiooniga, erandiks on 
#kaks riiki, mille puhul määrame koodi käsitsi:
def lisa_ISO(dfx):
    koodid=[]
    for riik in list(dfx.index.values): #veerunimede (riikide) list
        if riik=='Moldova':
            koodid.append('MD')
        elif riik=='US':
            koodid.append('US') 
        else:
            koodid.append(pycountry.countries.get(name=riik).alpha_2) #tuvastame ISO-koodi
    dfx["ISO_kood"]=koodid # lisame uue veeru tabelisse

lisa_ISO(df3)
print("Riike kokku:", len(df3)) #palju riike nüüd veel tabelisse on jäänud

df3.sort_values("points", ascending=False)

Nagu tabelist näha, on degustaatorid kõige kvaliteetsemaks pidanud Austria veine, millele tihedalt järgnevad Prantsusmaa, Saksamaa, Itaalia, Kanada, Sloveenia, Türgi ja Portugali veinid.

Kõige kohutavama kvaliteediga on aga Brasiilia veinid, millele järgnevad Uruguay, Molodva ja Mehhiko. 

Kvaliteet varieerub kõige rohkem USA ja Ungari, aga ka Prantsusmaa, Hisapaania, Argentiina ja Gruusia veinide puhul (nende standardhälve on üle 3 punkti). 

Kõige rohkem hinnanguid sisaldab see andmebaas USA veinide kohta.


Tulemuste visualiseerimiseks teeme graafiku riikidest ja nende veinidele antud keskmistest hinnangutest:

In [ ]:
#TULPDIAGRAMM:
def joonista(dfx, veerg, ytelg_title, pealkiri, y_max, y_min):
    dfx=dfx.sort_values(veerg, ascending=True)
    labels = dfx['ISO_kood'] #tulpade nimed
    data   = dfx[veerg] #andmed
    y_pos = range(len(labels)) #tulpade järjekord (mis järjekorras kuvatakse)

    plt.figure(figsize=(20,10))
    plt.bar(y_pos, data, color=matplotlib.colors.to_hex("#3399ff", keep_alpha=False), width=0.6)
    #värvi valimiseks saab kasutada värvide hex-koode: https://www.w3schools.com/colors/colors_picker.asp
    plt.xticks(y_pos, labels, fontsize=20, fontweight='bold') # tulpade nimed  #rotation=90, 
    plt.yticks(fontsize=20, fontweight='bold')
    plt.ylim(ymax = y_max, ymin = y_min) #y-telje väärtuste vahemik

    # joonise ja telgede nimed:
    plt.title(pealkiri, size=30, fontweight='bold')
    plt.xlabel('\nRIIGID', size=24, fontweight='bold')
    plt.ylabel(ytelg_title+"\n", size=24, fontweight='bold')

    # kuvame joonise:
    plt.show()

joonista(df3, "points", "HINNANGUD", "VEINIDELE ANTUD HINNANGUD", 90, 80)

## **4.2 Milline on veinide keskmine hind erinevates riikides?**

Selle uurimiseks koostame eelmisega sarnase tabeli ka erinevate riikide veinihindande kohta. 

Kuigi Kaggle'i andmestiku kirjelduses ei ole seda öeldud, võib arvata, et veini hinnad on toodud dollarites, sest andmestiku üleslaadija (Kaggle'i kasutaja kasutajanimega zackthoutt) on pärit USA-st. See, et hinnad on samas valuutas, teeb analüüsi lihtsamaks, sest andmeid ei ole vaja eri valuutadest ümber arvutada. 



In [ ]:
df4=pd.DataFrame(df.groupby("country")["price"].mean())
#df4.sort_values("price", ascending=False)

#Veergude lisamine:
df4["max_price"]=pd.DataFrame(df.groupby("country")["price"].max())
df4["min_price"]=pd.DataFrame(df.groupby("country")["price"].min())
df4["max-min_price"]=df4["max_price"]-df4["min_price"] #lahutame ühe Series objekti teisest
df4["stdev_price"]=pd.DataFrame(df.groupby("country")["price"].std())
#df4["var_price"]=pd.DataFrame(df.groupby("country")["price"].var())
df4["hinnanguid"]=df["country"].value_counts()

##Filtreerime:
df5=df4[df4.hinnanguid>=20]

lisa_ISO(df5) #lisame ISO-koodide veeru

print("Riike kokku:", len(df5)) #palju riike nüüd veel tabelisse on jäänud
df5=df5.sort_values("price", ascending=False) 
df5

Tabelist näeme, et keskmiselt kõige kallimad degusteeritud veinid olid Prantsusmaal (45.62 USD), millele üllatuslikult järgnes Ungari (44.20 USD), ning seejärel Saksamaa (39.01 USD) ja Itaalia (37.55 USD). Samas kõikusid Prantsusmaa veinide hinnad ka kõige enam. 

Seevastu keskmiselt kõige odavamad olid Bulgaaria, Moldova ja Küprose veinid (vastavalt 11.55 USD, 15.37 USD ja 15.48 USD), mille puhul ka keskmised hinnaerinevused olid riikidest kõige väiksemad.

Hinnatud veinide seast üldse kõige madalama hinnaga olid 4.00 USD maksnud veinid. Sellised veinid olid pärit USA-st, Argentiinast ja Rumeeniast. Kõige kallim hinnatud vein (2300 USD) pärines aga Prantsusmaalt. Samas Prantsusmaa odavaimad degustaatorite poolt hinnatud veinid maksid vaid 5.00 USD.  

Joonistame tulpdiagrammi ka riikide keskmise hinnataseme kohta:

In [ ]:
joonista(df5, "price", "HINNAD", "VEINIDE KESKMINE HIND RIIKIDE LÕIKES", 50, 10)

Nagu histogrammilt näha, on riikide hinnataseme erinevused märksa suuremad kui seda olid kvaliteedi erinevused.

## **KÕIGE KALLIM VEIN:**

Leiame ka täpsema info kõige kallima veini kohta:



In [ ]:
max_price=df["price"].max()

df6=df.loc[[indeks for indeks in range(len(df)) if df["price"][indeks]==max_price]] 
print('KALLEIMA VEINI KIRJELDUS:', df6["description"].tolist())
df6

Kalleim vein on pärit Bodeaux'st, Château Latour'i viinamäelt ning maksab 2300.00 USD. 

In [ ]:
#ODAVAIMAD VEINID:
#min_price=df["price"].min()
#df.loc[[indeks for indeks in range(len(df)) if df["price"][indeks]==min_price]] 

 ## **4.3 Miline on veinide keskmine hinna/kvaliteedi suhe erinevates riikides?**

Meie kolmas ja viimane uurimisküsimus oli, milline on veinide keskmine hinna ja kvaliteedi suhe riikide lõikes.

Selle uurimiseks koondame keskmised hinnad ja keskmised punktiskoorid ühte tabelisse ning kujutame tulemused tulpdiagrammil ja scatterplotil.


In [ ]:
#Kontrollime, kas hindade ja punktide tabeli reaindeksid kattuvad või on erinevusi:
#sorted(list(df3.index.values))==sorted(list(df6.index.values))  #True -> kattuvad

#Sorteerime dataframe'id, et nende indeksid oleksid samas järjekorras:
df3=df3.sort_index(ascending=True)
df5=df5.sort_index(ascending=True)

#Teeme uue dataframe'i kahe olemasoleva kolmest veerust:
data=[df3["points"], df5["price"], df3["ISO_kood"]]
a = pd.concat(data, axis=1, keys=[s.name for s in data])

#Lisame hinna ja kvaliteedi suhte veeru:
a["hinna-kvaliteedi suhe"]=round(a["points"]/a["price"], 2) 

a=a.sort_values("hinna-kvaliteedi suhe", ascending=False) 
a

Tulpdiagrammina:

In [ ]:
joonista(a, "hinna-kvaliteedi suhe", "HINNA JA KVALITEEDI SUHE", "VEINIDE KESKMINE HINNA JA KVALITEEDI SUHE RIIKIDE LÕIKES", 9, 0)

Teeme hinna ja kvaliteedi suhte kohta scatterploti:

In [ ]:
#SCATTERPLOT:

hinnad=a['price'].values.tolist()
punktid=a['points'].values.tolist()
n=list(a.index.values)

#plt.figure(figsize=(20,10))

fig, ax = plt.subplots(figsize=(20,10))
ax.scatter(punktid, hinnad, color=matplotlib.colors.to_hex("#ff66ff", keep_alpha=False), s=400)

plt.yticks(fontsize=20, fontweight='bold')
plt.xticks(fontsize=20, fontweight='bold')
#plt.ylim(ymax = y_max, ymin = y_min) #y-telje väärtuste vahemik



# joonise ja telgede nimed:
plt.title("HINNA JA KVALITEEDI SUHE\n", size=30, fontweight='bold')
plt.xlabel('\nKVALITEET', size=24, fontweight='bold')
plt.ylabel("HIND\n", size=24, fontweight='bold')

for i, txt in enumerate(n):
    ax.annotate(txt, (punktid[i],hinnad[i]), fontsize=20, fontweight='bold')



Jooniselt on näha, et hind ja kvaliteet on omavahel üldjoontes positiivses korrelatsioonis: mida kõrgem on hind, seda kõrgem on ka kvaliteet. Samas on riikide vahel suured erinevused: nt Austria veinid on küll kõige kvaliteetsemad, aga üldse mitte kõige kallimad; sama kvaliteediga Ungari veini eest tuleb  aga välja käia kaks korda rohkem kui Lõuna-Aafrika Vabariigi oma eest. 